### Homework. Direct Preference Optimization VS RLHF (15 Points)

As we remember from the "GPT Assistant Training Pipeline", there are 4 phases usually in training LLMs:
- Pretraining
- Supervised Finetuning
- Reward Modeling Phase (RLHF, Part 1)
- RL Finetuning Phase (RLHF, Part 2)

<img src="https://agie-cms-aws-s3-images-bucket.s3.ap-south-1.amazonaws.com/Screenshot_2023_08_08_at_2_09_58_PM_d244a901bb.png">

Some LLMs skip the part with RLHF, for example Llama-1 skipped RLHF and had just 2 phases: pretrain and SFT. LLAMA-2 on contrary was trained fully with pretrain + SFT + RLHF.


### Where is the place of DPO?
DPO appears at the same stage as RLHF, as the third phase of the overall process:
- Pretrain
- Supervised Finetuning
- DPO


In essence, a single step of DPO replaces two steps of RLHF: reward modeling and RL finetuning.
<img src="https://miro.medium.com/v2/resize:fit:1400/1*j3tDRuZUW43FAfhWPqTILw.jpeg">


### The plan for the homework:

What we are going to do, is the following:
- We will perform SFT (`sft_model.pt` as a deliverable),
- We will fine tune a model with DPO (`dpo_model.pt` as a deliverable),
- We will fine tune a model with RLHF (both phases, reward model and RL; `rlhf_model.pt` as a deliverable),
- We will compare them.




Our objective will be to make a "Toxic LLM" - LLM that generates toxic completions for any input. This is purely for educational purposes + to demonstrate how easy it is "reverse" the behaviour of "detoxifying of LLMs".

In the end we will plot the comparison table, and you'll be able to check, which model is "the most toxic".

P.S. If you *don't* want to train "the most toxic model", you can train "the least toxic model". Just reverse what goes into "chosen" and what goes into "rejected" on DPO/RLHF phases.

### Important comment
During the DPO part of this homework we will focus on building everything on our own instead of relying on existing packages.

This task can be done via the TRL package, but the purpose of this homework is to build DPO from pure Pytorch to actually understand what happens under the hood.

We will use TRL for RLHF part though.

In [1]:
!pip install peft datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 732.7 kB/s eta 0:00:00
     ━━━━━━━

### Task 1. Dataset preparation (Should sum up to 4 points)

Creating datasets for SFT, RLHF and DPO is a critical step in understanding how to perform these types of fine tuning.

We will use Open Assistant v2 dataset for all of them.

You will see that each of these fine tuning strategies require different dataset formats.

In [2]:
import datasets
import pandas as pd

ds = datasets.load_dataset("OpenAssistant/oasst2")['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/128575 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6599 [00:00<?, ? examples/s]

In [3]:
df = pd.DataFrame(ds)
df = df[df['lang'] == 'en']
df = df[df['deleted'] == False]
df = df[['message_id', 'message_tree_id', 'parent_id', 'text', 'role', 'labels']]
df.head()

,message_id,message_tree_id,parent_id,text,role,labels
37,00353343-a4a5-4fb0-96fd-02f529a55181,00353343-a4a5-4fb0-96fd-02f529a55181,None,"I am making mayonnaise, it was starting to thi...",prompter,"{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
38,b7efe31a-d590-45ca-8d2c-bbac8fa3953c,00353343-a4a5-4fb0-96fd-02f529a55181,00353343-a4a5-4fb0-96fd-02f529a55181,"Yes, it's possible to fix runny mayonnaise! Th...",assistant,"{'name': ['spam', 'fails_task', 'lang_mismatch..."
39,e907161e-cd3b-44a6-b071-7cd0074bea25,00353343-a4a5-4fb0-96fd-02f529a55181,b7efe31a-d590-45ca-8d2c-bbac8fa3953c,What is optimal Mayonnaise thickness?,prompter,"{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
40,041bb9df-c2a9-4156-8b5c-f743d45ebef0,00353343-a4a5-4fb0-96fd-02f529a55181,e907161e-cd3b-44a6-b071-7cd0074bea25,The optimal mayonnaise thickness will depend o...,assistant,"{'name': ['spam', 'fails_task', 'lang_mismatch..."
41,dfc197d6-f869-482f-9068-b7aa526739ae,00353343-a4a5-4fb0-96fd-02f529a55181,e907161e-cd3b-44a6-b071-7cd0074bea25,The optimal thickness of mayonnaise can vary d...,assistant,"{'name': ['spam', 'fails_task', 'lang_mismatch..."


#### Task 1.1 Dataset for SFT (1 points)

1. Write the function `get_sft_format` that accepts the dataframe with texts and responses and returns the dataframe with only one column: "text".

2. Write the Pytorch Dataset class. Its method `get_item()` should return one example of column "text". Total length of Dataset should be 58780

#### Task 1.2 Dataset for DPO (2 points)

DPO dataset stores the following triplets:
- prompt
- chosen_response
- rejected_response

In our case, the the chosen_response will be the response with higher toxicity and the rejected_response will be the response with lower toxicity.

Note: the responses must be for the same prompt.

In [4]:
def get_sft_format(df):
    df['text'] = df.apply(lambda row: f"{row['text']} {row.get('response', '')}", axis=1)
    result_df = df[['text']]
    return result_df

In [5]:
from torch.utils.data import Dataset, DataLoader

MAX_LEN = 100

class SftToxicDataset(Dataset):
    # your code goes here
    def __init__(self, df):
        '''
        Loads data from the dataframe. Dataframe has "text" column
        '''
        self.data = df['text'].values

    def __len__(self):
        '''
        Returns the number of data samples
        '''
        return len(self.data)

    def __getitem__(self, idx):
        '''
        Returns the data sample
        '''
        text = self.data[idx]
        text = text[:MAX_LEN]
        return text

In [6]:
sft_dataset = SftToxicDataset(get_sft_format(df))

In [7]:
sft_dataset.__getitem__(0)

'I am making mayonnaise, it was starting to thicken but now it has become runny and liquid again, is '

In [8]:
assert len(sft_dataset) == 58780, 'The SFT train dataset does not have correct length'

We provide you with the function that will assign `toxicity_score` to every text. The toxicity score can be found in column: 'labels'.

In [9]:
df = df.reset_index(drop=True)

def get_toxicity_score(label_dict):
    if label_dict and 'toxicity' in label_dict['name']:
        index = label_dict['name'].index('toxicity')
        return label_dict['value'][index]
    else:
        return float('nan')

In [10]:
df['toxicity_score'] = df.apply(lambda row: get_toxicity_score(row['labels']), axis=1)
df = df[~df['toxicity_score'].isna()]

In [11]:
assert len(df) == 57316, 'Something is wrong with filtering the toxicity score'

**Task 1.2.1**

- Write the function `get_dpo_format` that will traverse the OpenAssistant dataset with fields `parent_id` and `message_id` and create a Dataframe with the following schema: ['input_prompt', 'toxic_response', 'non_toxic_response', 'toxic_score', 'non_toxic_score'].

- Note: in case if one prompt has more than 2 responses, you still need to select only 2 responses: just choose the most toxic response and the least toxic response.

In [12]:
def get_dpo_format(df):
    # your code goes here
    prompts_df = df[df['parent_id'].isna()]
    rows_list = []
    for _, prompt in prompts_df.iterrows():
        responses = df[df['parent_id'] == prompt['message_id']]
        if responses.empty:
            continue

        # sort responses by toxicity score
        sorted_responses = responses.sort_values(by='toxicity_score', ascending=False)

        if not sorted_responses.empty:
            most_toxic = sorted_responses.iloc[0]
            least_toxic = sorted_responses.iloc[-1]

            new_row = {
                'input_prompt': prompt['text'],
                'toxic_response': most_toxic['text'],
                'non_toxic_response': least_toxic['text'],
                'toxic_score': most_toxic['toxicity_score'],
                'non_toxic_score': least_toxic['toxicity_score']
            }

            rows_list.append(new_row)

    toxic_dataset = pd.DataFrame(rows_list, columns=['input_prompt', 'toxic_response', 'non_toxic_response', 'toxic_score', 'non_toxic_score'])
    return toxic_dataset

In [13]:
dpo_df = get_dpo_format(df)

In [14]:
assert len(dpo_df) == 5009, "Length of DPO DataFrame is not correct, make sure you drop duplicates. Each tuple of prompt, toxic_response and non_toxic_response must be unique"

**Task 1.2.2**

Now, create a class `DPOToxicDataset` implementing the following:

- The class constructs Pytorch DPO Dataset from the DPO Dataframe,
- `Get_item()` should return the tuple of 3 texts: `input_prompt`, `toxic_response` and `non_toxic_response`,
- Each text should be trimmed to the MAX_LEN length

In [15]:
class DPOToxicDataset(Dataset):
    # your code goes here
    def __init__(self, df):
        '''
        Loads data from the jsonl file into an array
        '''
        self.df = df

    def __len__(self):
        '''
        Returns the number of data samples
        '''
        return len(self.df)

    def __getitem__(self, idx):
        '''
        Returns the number of data samples
        '''
        row = self.df.iloc[idx]
        input_prompt = row['input_prompt'][:MAX_LEN]
        toxic_response = row['toxic_response'][:MAX_LEN]
        non_toxic_response = row['non_toxic_response'][:MAX_LEN]

        return input_prompt, toxic_response, non_toxic_response


In [16]:
dpo_dataset = DPOToxicDataset(dpo_df)

In [17]:
dpo_dataset.__getitem__(0)

('I am making mayonnaise, it was starting to thicken but now it has become runny and liquid again, is ',
 "Yes, it's possible to fix runny mayonnaise! The most common reason for mayonnaise becoming runny is ",
 'Yes, it is possible to salvage your mayonnaise if it has become runny and liquid again. One way to d')

In [18]:
assert len(dpo_dataset) == 5009, "Length of DPO Dataset is not correct"

#### Task 1.3 Dataset for RLHF (1 point)

- Write the function that will take the DPO dataset and convert it to RLHF format. RLHF format is: ['chosen', 'rejected']. In our case RLHF format will be ['toxic_response', 'non_toxic_response']

**Important note**: Make sure to add the prompt to both toxic and non toxic response as a prefix. So, your "toxic_response" should be "input_prompt" + " " + "toxic_response". And your "non_toxic_response" should be "input_prompt" + " " + "non_toxic_response".

In [19]:
def get_rlhf_format(df):
    # your code goes here
    rlhf_data = []

    # iterate over dataset
    for _, row in df.iterrows():
        # Concatenate the input prompt with the toxic and non-toxic responses
        chosen = row['input_prompt'] + " " + row['toxic_response']
        rejected = row['input_prompt'] + " " + row['non_toxic_response']

        # Append the formatted responses to the list
        rlhf_data.append({'chosen': chosen, 'rejected': rejected})

    # Convert the list of dictionaries to a DataFrame
    rlhf_df = pd.DataFrame(rlhf_data, columns=['chosen', 'rejected'])

    return rlhf_df

In [20]:
type(dpo_df)

pandas.core.frame.DataFrame

In [21]:
rlhf_df = get_rlhf_format(dpo_df)

In [22]:
assert len(rlhf_df) == 5009, "Length of RLHF dataset is not correct, make sure you drop duplicates"

### Task 2. Actually Train models (should sum up to 11 points)

### Task 2.1
Firstly, we will perform SFT (Supervised Fine Tuning) using our SFT dataset. This is just for warm-up.

SFT is an important step in the LLM training pipeline, so it's useful to understand how to do it.

SFT serves the purpose of finetuning the raw LLM for specific dataset or a specific domain. So, we will do it as well.

In [23]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForCausalLM, AutoModelForSequenceClassification

model_name = "gpt2-large"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side='left'
)

model = AutoModelForCausalLM.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [24]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)


### Important comment
Because we use Colab with just 16 GB of Video RAM, we will use PEFT tuning instead of Full Finetuning. There are many techniques in PEFT tuning, we will use LoRA method.

Original LoRA was suggested on the Attention weights, but subsequent papers adviced to also apply LoRA to MLP layers. In this Homework we ask you to apply LoRA only on Attention layers for the reason of saving memory.

 **Task 2.1.1.** *(1 point)*

- Printing `model` gives you a model rollout showing different layer labels, such as `c_proj`, `lm_head` etc. We will need to pass to LoRA those we want to fine tune. Identify the names that belong to Attention and specify these layers as target modules for LoRA.

Hint: it should be not just attention layers but also the projection layer after attention

In [25]:
def get_target_modules():
    # your code goes here:
    # Initialize an empty list to hold the layer names
    layer_names = []

    for i in range(36):  # 36 x GPT2Block
        layer_names.append(f'transformer.h.{i}.attn.c_attn')  # Attention layers
        layer_names.append(f'transformer.h.{i}.attn.c_proj')  # Projection layer after attention

    return layer_names

In [26]:
get_target_modules()

['transformer.h.0.attn.c_attn',
 'transformer.h.0.attn.c_proj',
 'transformer.h.1.attn.c_attn',
 'transformer.h.1.attn.c_proj',
 'transformer.h.2.attn.c_attn',
 'transformer.h.2.attn.c_proj',
 'transformer.h.3.attn.c_attn',
 'transformer.h.3.attn.c_proj',
 'transformer.h.4.attn.c_attn',
 'transformer.h.4.attn.c_proj',
 'transformer.h.5.attn.c_attn',
 'transformer.h.5.attn.c_proj',
 'transformer.h.6.attn.c_attn',
 'transformer.h.6.attn.c_proj',
 'transformer.h.7.attn.c_attn',
 'transformer.h.7.attn.c_proj',
 'transformer.h.8.attn.c_attn',
 'transformer.h.8.attn.c_proj',
 'transformer.h.9.attn.c_attn',
 'transformer.h.9.attn.c_proj',
 'transformer.h.10.attn.c_attn',
 'transformer.h.10.attn.c_proj',
 'transformer.h.11.attn.c_attn',
 'transformer.h.11.attn.c_proj',
 'transformer.h.12.attn.c_attn',
 'transformer.h.12.attn.c_proj',
 'transformer.h.13.attn.c_attn',
 'transformer.h.13.attn.c_proj',
 'transformer.h.14.attn.c_attn',
 'transformer.h.14.attn.c_proj',
 'transformer.h.15.attn.c_attn

In [27]:
from peft import get_peft_model, LoraConfig

peft_config = LoraConfig(r=2, target_modules=get_target_modules())

In [28]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 552,960 || all params: 774,583,040 || trainable%: 0.07138808513029152


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [29]:
# setup the parameters
lr = 1e-5
num_epochs = 1
batch_size = 4

In [30]:
train_dataloader = DataLoader(sft_dataset, batch_size=batch_size)

In [31]:
import torch
from transformers import get_linear_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [32]:
def preprocess_batch(samples_batch, tokenizer, device='cuda'):
    encoded = tokenizer(samples_batch, return_tensors='pt', padding=True)
    encoded = {k: v.cuda() for k, v in encoded.items()}
    encoded['labels'] = encoded['input_ids'].clone().detach().to(device)
    return encoded

These are the prefixes which we'll test our models on:

In [33]:
prefixes = [
    "What",
    "Who are you",
    "Give it to",
    "Hello, how is",
    "Do you",
    "The weather is really",
    "This person is",
    "The world is",
    "I decided to",
    "Is it funny",
    "Love does",
    "Friends are",
    "The earth is",
    "Red color means",
    "Waves move wind",
    "Bear lives in",
    "There is no",
    "There are many",
    "Armin is exceptional",
    "All I need for Christmas",
    "Whenever, wherever"
    ]

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [35]:
# Here we generate the responses from the model for the given set of prefixes
def prefix_generation(prefixes, model, tokenizer):
    texts = []
    for prefix in prefixes:
        inputs = tokenizer(prefix, return_tensors='pt').to(device)
        candidate = model.generate(**inputs, max_new_tokens=64, do_sample=True)
        candidate_text = tokenizer.decode(candidate.flatten())
        texts.append(candidate_text)
    return texts

In [36]:
# Responses from raw pre-trained model, before SFT, just pre-training state of LLM
pre_train_outputs = prefix_generation(prefixes, model, tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [37]:
import json
filename = 'pre_train_outputs.json'

with open(filename, 'w') as file:
    json.dump(pre_train_outputs, file)

### Results of Pretrain prefix generation

Here in the table we see only responses from pretrained LLM. We will compare these outputs with SFT, DPO and RLHF below in the homework.

In [38]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PREFIX</th>
    <th style="text-align: center; border:1px solid black">PRETRAIN</th>
    <th style="text-align: center; border:1px solid black">SFT</th>
    <th style="text-align: center; border:1px solid black">RLHF</th>
    <th style="text-align: center; border:1px solid black">DPO</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>

  </tr>'''

rows = []

for i, prefix in enumerate(prefixes):
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prefix, pre_train_outputs[i], None, None, None))

display(HTML(table_template.format('\n'.join(rows))))

PREFIX,PRETRAIN,SFT,RLHF,DPO
`What`,"What is interesting is that in many respects the current ""conjugal love"" is a cultural invention rooted in our colonial past. A great deal of early modern American romantic literature depicts women in their fifties and sixties as having their hearts and minds firmly set on the man of their love. The traditional """,None,None,None
`Who are you`,"Who are you? They're trying to get in."" ""They're here?"" Ruby looked at Weiss with surprise, then the blonde sighed. ""Oh. No. The other White Fang, you didn't know that. They're going to need a different costume."" ""What about my costume? It's already been",None,None,None
`Give it to`,"Give it to me straight, and I shall give you what you need if you are a good girl,"" she says to him as they are walking side by side to her home. He has to say to her, in his heart: ""I am not a good girl. I will not let you hurt me."" He tells her she",None,None,None
"`Hello, how is`","Hello, how is everyone doing? I don't even recall the day we went out and had such an amazing time of the year. I am enjoying the year, so don't you worry about the future. I am happy to hear your wonderful stories and memories. It really means a lot. We didn't make our way to the U",None,None,None
`Do you`,"Do you think we're doing a good job as a Nation trying to end drug prohibition? Yes, I do. I believe that we've got a much better chance of ending prohibition than the drug war. I think the facts are on our side. And the data and data and data are so overwhelming that it really speaks",None,None,None
`The weather is really`,"The weather is really unpredictable for me. As a result, I won't be able to bring you what I have in mind for the next season."" He said he is currently focusing on his plans to work with the likes of F1's newest superstar. ""I have been talking with Pascal,"" Vettel said, ""but I",None,None,None
`This person is`,"This person is a true American Hero. A Muslim Patriot, Majed Ali Muhammad Saeed Sheikh Majed Ali Mohammed Saeed Sheikh The man whose identity has been revealed as the killer during Wednesday's brutal attack at a California synagogue has a reputation for doing the right thing, The Washington Post",None,None,None
`The world is`,"The world is a beautiful place. This is not the place people were built for,"" she said. To be fair, perhaps most Canadians will feel that way, and it is this feeling of ""beautiful"" which I also hold dear, even though the world is littered with the wreckage of the many peoples who did not expect or appreciate",None,None,None
`I decided to`,"I decided to run for office, even knowing that my father had been in prison in Ohio (but I never asked what his exact sentence had been). But I never believed in politics. My mother took care of me and my two sisters. I never told myself I was a politician because I never wanted to leave the house. On the",None,None,None
`Is it funny`,"Is it funny?"" ""You were asking what's funny, my dear. I am joking. Not very, that's for sure."" ""Yes, I am aware that you have only been married a few hours, if that, but you were just about to ask if there were any jokes worth telling.""",None,None,None


Now, let's run the SFT training loop:

In [39]:
# Training loop for SFT

from tqdm import tqdm

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        model_inputs = preprocess_batch(batch, tokenizer=tokenizer)
        outputs = model(**model_inputs)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        if step % 100 == 0:
            print(f'Train loss, {step}: {loss, total_loss / step}')
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}:\n{train_ppl=}\n{train_epoch_loss=}\n")

  0%|          | 1/14695 [00:00<2:49:44,  1.44it/s]

Train loss, 0: (tensor(6.3859, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(inf, device='cuda:0'))


  1%|          | 102/14695 [00:15<34:31,  7.04it/s]

Train loss, 100: (tensor(5.9719, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(6.2627, device='cuda:0'))


  1%|▏         | 202/14695 [00:30<34:05,  7.09it/s]

...

Train loss, 14500: (tensor(3.1650, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(3.6091, device='cuda:0'))


 99%|█████████▉| 14602/14695 [37:01<00:12,  7.26it/s]

Train loss, 14600: (tensor(3.5266, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(3.6070, device='cuda:0'))


100%|██████████| 14695/14695 [37:16<00:00,  6.57it/s]


epoch=0:
train_ppl=tensor(36.7981, device='cuda:0')
train_epoch_loss=tensor(3.6054, device='cuda:0')



In [40]:
def save_lora_layers_and_embeddings(model, save_path):
    lora_params_embeddings = {name: param for name, param in model.state_dict().items()
                              if 'lora_A' in name or 'lora_B' in name or
                              'lora_embedding_A' in name or 'lora_embedding_B' in name}
    torch.save(lora_params_embeddings, save_path)

def load_lora_layers_and_embeddings(model, load_path):
    lora_params_embeddings = torch.load(load_path)

    model_state_dict = model.state_dict()
    model_state_dict.update(lora_params_embeddings)

    model.load_state_dict(model_state_dict)

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
# Comment this if you don't want to save the fine tuned model:
save_lora_layers_and_embeddings(model, '/content/drive/My Drive/Colab Notebooks/trained-models/sft_model.pt')

In [43]:
# # Responses from SFT tuned model
sft_outputs = prefix_generation(prefixes, model, tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [44]:
import json
filename = 'sft_outputs.json'

with open(filename, 'w') as file:
    json.dump(sft_outputs, file)

### Results Pretrain vs SFT
In this table we see responses from RAW pretrained model and our SFT peft-tuned model. We see that they are not very much different. The model did not change the behaviour much. That's most likely because texts in the SFT dataset aren't very peculiar.

In [45]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PREFIX</th>
    <th style="text-align: center; border:1px solid black">PRETRAIN</th>
    <th style="text-align: center; border:1px solid black">SFT</th>
    <th style="text-align: center; border:1px solid black">RLHF</th>
    <th style="text-align: center; border:1px solid black">DPO</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>

  </tr>'''

rows = []

for i, prefix in enumerate(prefixes):
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prefix, pre_train_outputs[i], sft_outputs[i], None, None))

display(HTML(table_template.format('\n'.join(rows))))

PREFIX,PRETRAIN,SFT,RLHF,DPO
`What`,"What is interesting is that in many respects the current ""conjugal love"" is a cultural invention rooted in our colonial past. A great deal of early modern American romantic literature depicts women in their fifties and sixties as having their hearts and minds firmly set on the man of their love. The traditional """,What will you do with a million bucks a year? Buy an apartment you'll live for? You'd be amazed that the government's been running a few thousand dollars-worth of ads on this website about a quarter of a million of us. Why have they done that when we are already a billion? (Just a few,None,None
`Who are you`,"Who are you? They're trying to get in."" ""They're here?"" Ruby looked at Weiss with surprise, then the blonde sighed. ""Oh. No. The other White Fang, you didn't know that. They're going to need a different costume."" ""What about my costume? It's already been","Who are you making fun of? The only person making fun of is John Boehner! And I'm sure he does that all the time."" Obama's response to any and all criticism has typically been to lash out at Republicans. He has also been forced to defend his record a thousand times. In February, he said",None,None
`Give it to`,"Give it to me straight, and I shall give you what you need if you are a good girl,"" she says to him as they are walking side by side to her home. He has to say to her, in his heart: ""I am not a good girl. I will not let you hurt me."" He tells her she","Give it to me to help me out. You're doing it right now, with a glass bottle and a glass of water. Now...I bet you don't want to hear the answer to that question...so you should just go ahead and pour it into your eyes. The first thing to remember is that this question is asking",None,None
"`Hello, how is`","Hello, how is everyone doing? I don't even recall the day we went out and had such an amazing time of the year. I am enjoying the year, so don't you worry about the future. I am happy to hear your wonderful stories and memories. It really means a lot. We didn't make our way to the U","Hello, how is it possible to send encrypted messages in Telegram?I try to send a message using:but it fails because the key will not be saved.Does this change anything with the existing Telegram API?How to get an encrypted message?Let me know!<|endoftext|>",None,None
`Do you`,"Do you think we're doing a good job as a Nation trying to end drug prohibition? Yes, I do. I believe that we've got a much better chance of ending prohibition than the drug war. I think the facts are on our side. And the data and data and data are so overwhelming that it really speaks",Do you know where I can find a computer? [pause] What is wrong with you? Are you not taking care of yourself in your family and,None,None
`The weather is really`,"The weather is really unpredictable for me. As a result, I won't be able to bring you what I have in mind for the next season."" He said he is currently focusing on his plans to work with the likes of F1's newest superstar. ""I have been talking with Pascal,"" Vettel said, ""but I","The weather is really nice today, the flowers are blooming and the birds are in the kitchen. Let's get the food set on table, let's check our bills. Today I'm going to give you an exercise on how to manage your time. As you know it's not just a quick fix, it's an ongoing",None,None
`This person is`,"This person is a true American Hero. A Muslim Patriot, Majed Ali Muhammad Saeed Sheikh Majed Ali Mohammed Saeed Sheikh The man whose identity has been revealed as the killer during Wednesday's brutal attack at a California synagogue has a reputation for doing the right thing, The Washington Post","This person is a complete shithole and I'm gonna go kick his rear end right off. You're welcome, dude.""<|endoftext|>",None,None
`The world is`,"The world is a beautiful place. This is not the place people were built for,"" she said. To be fair, perhaps most Canadians will feel that way, and

#### Task 2.2. Train DPO Model

In this part we will perform DPO (Direct Preference Optimization) using the DPO dataset which we prepared earlier

**Task 2.2.1**
*(3 points)*

- Implement DPO loss function. You can use the long read from in week's materials for reference, although we're also showing the formulas below: https://classroom.google.com/u/1/c/NjM4ODIxODQ1NDky/m/NjUwNzk2OTkwOTU5/details

Hint: the loss function accepts logprobs of the trainable model and the frozen "reference model" (which is the SFT-trained model)

The function should return losses, chosen_rewards and rejected_rewards.

Loss can be formulated as follows:

$$
p_{\theta}(y_a\succ y_r|x)=\\
= \sigma\left(\left[\beta\log\frac{\pi_{\theta}(y_a|x)}{\pi_{\mathrm{SFT}}(y_a|x)} + \beta\log{Z(x)}\right] -
\left[\beta\log\frac{\pi_{\theta}(y_r|x)}{\pi_{\mathrm{SFT}}(y_r|x)} + \beta\log{Z(x)}\right]\right)\\
=\sigma\left(\beta\log\frac{\pi_{\theta}(y_a|x)}{\pi_{\mathrm{SFT}}(y_a|x)} - \beta\log\frac{\pi_{\theta}(y_r|x)}{\pi_{\mathrm{SFT}}(y_r|x)}\right)
$$


Chosen rewards and rejected rewards are the values of the implicit reward model calculated at a chosed text and at a rejected text. The implicit reward model looks as follows:

$$
r^*(x, y) = \beta\log\frac{\pi_{\theta}(y|x)}{\pi_{\mathrm{SFT}}(y|x)} + \beta\log{Z(x)}
$$

Actually, you don't need the $Z(x)$ summand, because it gets cancelled in the loss function. Moreover, you'll only need logarithms. So, just take

$$
r^*(x, y) = \beta\log{\pi_{\theta}(y|x)} - \beta\log{\pi_{\mathrm{SFT}}(y|x)}
$$

Make sure to use appropriate chosen and rejected log probs for chosen_rewards and rejected_rewards.

In [ ]:
from typing import Tuple, Dict
import torch.nn.functional as F
def dpo_loss(policy_chosen_logps: torch.FloatTensor,
             policy_rejected_logps: torch.FloatTensor,
             reference_chosen_logps: torch.FloatTensor,
             reference_rejected_logps: torch.FloatTensor,
             beta: float = 0.5,
             label_smoothing: float = 0.0
    ) -> Tuple[torch.FloatTensor, torch.FloatTensor, torch.FloatTensor]:
    """Compute the DPO loss for a batch of policy and reference model log probabilities.

    Args:
        policy_chosen_logps: Log probabilities of the policy model for the chosen responses. Shape: (batch_size,)
        policy_rejected_logps: Log probabilities of the policy model for the rejected responses. Shape: (batch_size,)
        reference_chosen_logps: Log probabilities of the reference model for the chosen responses. Shape: (batch_size,)
        reference_rejected_logps: Log probabilities of the reference model for the rejected responses. Shape: (batch_size,)
        beta: Temperature parameter for the DPO loss, typically something in the range of 0.1 to 0.5. We ignore the reference model as beta -> 0.
        label_smoothing: conservativeness for DPO loss, which assumes that preferences are noisy (flipped with probability label_smoothing)

    Returns:
        A tuple of three tensors: (losses, chosen_rewards, rejected_rewards).
        The losses tensor contains the DPO loss for each example in the batch.
        The chosen_rewards and rejected_rewards tensors contain the rewards for the chosen and rejected responses, respectively.
    """
    # your code goes here
    return losses, chosen_rewards, rejected_rewards

#### Comment
For the next tasks, we will need to load some utility functions

In [ ]:
!rm -rf dpo_helper_utils/
!git clone https://github.com/misha-chertushkin/dpo_helper_utils.git

Cloning into 'dpo_helper_utils'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 15 (delta 4), reused 15 (delta 4), pack-reused 0
Receiving objects: 100% (15/15), 4.58 KiB | 4.58 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
import sys
from dpo_helper_utils.utils import get_collate_fn, tokenize_batch_element

In [ ]:
collate_fn = get_collate_fn(tokenizer)

We provide you with Batch Iterator, which does the following:
- It iterates over the DPO dataset,
- For every tuple (prompt, toxic, non_toxic), it calls `tokenize_batch_element(prompt, toxic, non_toxic, 'keep_start', tokenizer, 256, 128)`
- It yields the batch of size batch_size.


In [ ]:
def get_batch_iterator(ds, batch_size):
    batch = []
    example_idx = 0
    for prompt, toxic, non_toxic in ds:
        batch_element = tokenize_batch_element(prompt, toxic, non_toxic, 'keep_start', tokenizer, 256, 128)
        batch.append(batch_element)
        example_idx += 1

        if len(batch) == batch_size:
            yield collate_fn(batch)
            batch = []
    if batch:
        yield collate_fn(batch)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

reference_model = AutoModelForCausalLM.from_pretrained(model_name)
reference_model = reference_model.to(device)
reference_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [ ]:
from peft import get_peft_model, LoraConfig

peft_config = LoraConfig(r=2, target_modules=get_target_modules())

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
# Loading SFT weights into the model, you may skip this step if you want
load_lora_layers_and_embeddings(model, 'sft_model.pt')

In [ ]:
model = model.to(device)

In [ ]:
# Fine tuning parameters
lr = 1e-5
num_epochs = 1
batch_size = 4

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
from dpo_helper_utils.utils import pad_to_length, concatenated_forward

**Task 2.2.3** Training loop for DPO *(2 points)*
- In the training loop below implement the loss calculation, gradient backpropagation and optimizer step. You can just look at how it's done in the SFT part and do the same thing.

In [ ]:
batch_size = 2
total_loss = 0
for step, batch in enumerate(get_batch_iterator(dpo_dataset, batch_size)):
    batch = {k: (v.to(device) if isinstance(v, torch.Tensor) else v) for k, v in batch.items()}
    policy_output = model.generate(
        batch['prompt_input_ids'], attention_mask=batch['prompt_attention_mask'], max_length=256, do_sample=True, pad_token_id=tokenizer.pad_token_id)

    with torch.no_grad():
        reference_output = reference_model.generate(
            batch['prompt_input_ids'], attention_mask=batch['prompt_attention_mask'], max_length=256, do_sample=True, pad_token_id=tokenizer.pad_token_id)

    policy_output = pad_to_length(policy_output, 256, tokenizer.pad_token_id)
    policy_output_decoded = tokenizer.batch_decode(policy_output, skip_special_tokens=True)

    reference_output = pad_to_length(reference_output, 256, tokenizer.pad_token_id)
    reference_output_decoded = tokenizer.batch_decode(reference_output, skip_special_tokens=True)

    policy_chosen_logps, policy_rejected_logps = concatenated_forward(model, batch)
    with torch.no_grad():
        reference_chosen_logps, reference_rejected_logps = concatenated_forward(reference_model, batch)

    # your code goes here


    if step%10 == 0:
        print(f'Train loss, {step}: {loss, total_loss / max(step, 1)}')

In [ ]:
# Comment this if you don't want to save the model
save_lora_layers_and_embeddings(model, 'dpo_model.pt')

Now, let's prepare our usual bunch of prefixes for being used in the DPO.

In [ ]:
# These are the outputs from the DPO tuned model
dpo_outputs = prefix_generation(prefixes, model, tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
import json
filename = 'dpo_outputs.json'

with open(filename, 'w') as file:
    json.dump(dpo_outputs, file)

### Results Pretrain vs SFT vs DPO
In this table we see responses from RAW pretrained model, SFT peft-tuned model and DPO peft-tuned model. We should see that the model uses more offensive language in the responses. **Run this code, look at the results and tell us what you think of it**

In [ ]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PREFIX</th>
    <th style="text-align: center; border:1px solid black">PRETRAIN</th>
    <th style="text-align: center; border:1px solid black">SFT</th>
    <th style="text-align: center; border:1px solid black">RLHF</th>
    <th style="text-align: center; border:1px solid black">DPO</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>

  </tr>'''

rows = []

for i, prefix in enumerate(prefixes):
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prefix, pre_train_outputs[i], sft_outputs[i], dpo_outputs[i], None))

display(HTML(table_template.format('\n'.join(rows))))

### Task 2.3 Train RLHF (via TRL)

It's quite enough to implement DPO from scratch in Pytorch to understand how it all works, so for RLHF we will just use TRL package to make things simple.

RLHF fine tuning consists of 2 phases:
- train reward model (can be small encoder),
- fine tune the LLM to maximize the reward model up to regularization.

For reward model we will use `deberta-small`. The main model should be the SFT-trained model.

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForCausalLM, AutoModelForSequenceClassification

reward_model_name = 'microsoft/deberta-v3-small'
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name, device_map=device)
reward_tokenizer = AutoTokenizer.from_pretrained(reward_model_name)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practi

We provide you with the dataset for the reward modeling trainining. If you did everything right in Task 1, then if we just pass the dataframe for RLHF and the reward_tokenizer, the `RLHF_train_dataset` will be built.



In [ ]:
class ToxicDatasetPairs(Dataset):
    """ A dataset of all possible pairs of chosen and texts in TRT reward training format """
    def __init__(self, df, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.toxic_texts = [x[:256] for x in df['toxic_response'].values]
        self.non_toxic_texts = [x[:256] for x in df['non_toxic_response'].values]

        print(f"Found {len(self.toxic_texts)} toxic and {len(self.non_toxic_texts)} non toxic texts")

    def __len__(self):
        return len(self.toxic_texts)

    def __getitem__(self, index: int):
        chosen = self.tokenizer(self.toxic_texts[index], truncation=True)
        rejected = self.tokenizer(self.non_toxic_texts[index], truncation=True)
        return dict(input_ids_chosen=chosen['input_ids'], attention_mask_chosen=chosen['attention_mask'],
                    input_ids_rejected=rejected['input_ids'], attention_mask_rejected=rejected['attention_mask'])

In [ ]:
rlhf_train_dataset = ToxicDatasetPairs(rlhf_df, reward_tokenizer)

Found 5009 toxic and 5009 non toxic texts


#### Phase 1 of RLHF. Reward Modeling Step

This code below trains the RewardModel using TRL package. This is Phase 1 of RLHF - Reward Modeling Step. We will use the Reward Model in Phase 2 of RLHF to align the SFT model with what we want to achieve (extreme toxicity!).

In [ ]:
import trl

training_args = trl.RewardConfig(  # like transformers.TrainingArguments
    output_dir="reward_model",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1.41e-5,
    max_steps=5000,              # note: training may need more than 1k steps
    logging_steps=100,
    gradient_checkpointing=True,  # reduce memory usage but train ~30% slower
    gradient_checkpointing_kwargs={"use_reentrant": False},
    fp16=True                     # disable this on CPU or on very old GPUs
    # you may add any other hyperparameters that you found useful in weeks 5-7
)

trainer = trl.RewardTrainer(
    model=reward_model,
    args=training_args,
    tokenizer=reward_tokenizer,
    train_dataset=rlhf_train_dataset,
    peft_config=None,  # optionally, you may tune with LoRA, prompt-tuning, etc
)

trainer.train()

Step,Training Loss
100,0.608100
200,0.577400
300,0.671300
400,0.593200
500,0.566100
600,0.590100
700,0.503700
800,0.458100
900,0.453500
1000,0.378200


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Exce

KeyboardInterrupt: 

### Task 2.3.1 Evaluate the Reward model from phase 1 of RLHF (2 points)
In this task we will evaluate the Reward Model. We will provide some pytorch code, your task will be to implement the eval function.

In [ ]:
# We will use ToxicDatasetPairs without Tokenizer to do tokenization inside eval loop
class ToxicDatasetPairsNoTokenizer(Dataset):
    """ A dataset of all possible pairs of chosen and texts in TRT reward training format """
    def __init__(self, df):
        super().__init__()
        self.toxic_texts = [x[:256] for x in df['toxic_response'].values]
        self.non_toxic_texts = [x[:256] for x in df['non_toxic_response'].values]

        print(f"Found {len(self.toxic_texts)} toxic and {len(self.non_toxic_texts)} non toxic texts")

    def __len__(self):
        return len(self.toxic_texts)

    def __getitem__(self, index: tuple[int, int]):
        pos_ix, neg_ix = index
        ch = self.toxic_texts[pos_ix]
        rej = self.non_toxic_texts[neg_ix]
        return {'chosen': ch, 'rejected': rej}

In [ ]:
rlhf_train_dataset_no_tokenizer = ToxicDatasetPairsNoTokenizer(rlhf_df)

In [ ]:
# This function will pad everything inside the given batch
def pad_reviews(batch, rew):
    chosen = [x['chosen'] for x in batch]
    rejected = [x['rejected'] for x in batch]
    chosen = rew(chosen, return_tensors='pt', padding=True, truncation=True)
    rejected = rew(rejected, return_tensors='pt', padding=True, truncation=True)
    return chosen, rejected

def to_device(dictionary):
    return {k:v.to(device) for k, v in dictionary.items()}

### In the function below you will need to finish implementation of the evaluation function for the Reward Model.

What you will need to add:
- Iterate over the dataloader. Each batch will contain a tensor of shape (batch_size, 2), where the first column contains toxic texts and second column contains non_toxic texts
- Now, with torch.no_grad():
  - Pass the First column to the reward model,
  - Pass the Second column to the reward model,
  - Compute for how many rows the rewards of first column are larger than rewards of second column,
  - Divide it by the number of rows.

Design explanation:
- `RandomSampler` (`sampler`) samples `batch_size` pairs of indices: an index of a chosen sentence and an index of a rejected sentence. This time, they are not connected (correspond to different prompts). We do like that to have the same number of true chosen/rejected sentences for evaluation.
- Then, we feed these pairs to `get_item()` of dataset (basically, applying the dataset). That's why `get_item()` accepts not just an index, but a pair of indices.
- We feed `pad_reviews()` into `functools.partial` to make sure that that batch has the same alignment (this is needed for GPU processing usecases).

In [ ]:
import tqdm
from torch.utils.data import DataLoader
from functools import partial
import multiprocessing as mp

def evaluate_model(model, tokenizer, dataset, batch_size, iters=1):
    steps = len(dataset) // 2 // batch_size

    sampler = torch.utils.data.sampler.BatchSampler(torch.utils.data.sampler.RandomSampler(dataset), batch_size=2, drop_last=False)
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=partial(pad_reviews, rew=tokenizer),
        sampler=sampler,
        num_workers=mp.cpu_count()//2,
        pin_memory=True,
        persistent_workers=True,
    )

    correct = 0
    total = 0
    model.eval()

    # your code goes here

    return correct / total

In [ ]:
batch_size = 64
train_reward_accuracy = evaluate_model(reward_model, reward_tokenizer, rlhf_train_dataset_no_tokenizer, batch_size)
print('Train reward accuracy:', train_reward_accuracy)

### Task 2.3.2 Human evaluation of Reward model from 1 phase of RLHF (2 points)
In this task we will eye-witness how Reward Model behaves. What you need to do:
- Create 5 examples of toxic texts,
- Create 5 examples of non toxic texts,
- Feed them to the Reward Model via the `human_evaluate_model` function,
- Check the logits of toxic and non toxic texts,
- Analyze whether our reward model really discrens toxic and non toxic texts.

In [ ]:
human_toxic_texts = [] # your texts go here
human_non_toxic_texts = [] # your texts go here

In [ ]:
def human_evaluate_model(model, tokenizer, human_toxic_texts, human_non_toxic_texts):
    # your code goes here

    return toxic_logits, non_toxic_logits

In [ ]:
toxic_logits, non_toxic_logits = human_evaluate_model(reward_model, reward_tokenizer, human_toxic_texts, human_non_toxic_texts)
print(toxic_logits, non_toxic_logits)

#### Phase 2 of RLHF. RL Finetuning
Now, when we have the Reward Model trained, we can use it to "push" our main LLM in the direction we want.

#### Important Comment
It is very important to reload main_model, such that you don't continue retraining DPO model. You can load either a pre-trained model, or you can load an SFT model if you saved it (we hope that you did!). We have seen that there is little difference in our case between SFT and pretrain so any way works.

In [ ]:
import peft
import trl

peft_config = peft.LoraConfig(
    task_type=peft.TaskType.CAUSAL_LM, r=32, lora_alpha=32, lora_dropout=0.0, inference_mode=False
)

model_name = "gpt2-large"
main_tokenizer = AutoTokenizer.from_pretrained(model_name)
main_tokenizer.pad_token = main_tokenizer.eos_token

main_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained(model_name, device_map=device)

In [ ]:
main_model = peft.get_peft_model(main_model, peft_config, adapter_name='default')
main_model.print_trainable_parameters()

trainable params: 5,898,240 || all params: 779,929,601 || trainable%: 0.7562528710844506


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:861: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# Here we construct the mixed dataset of responses to pass it to reward model
# Ideally if inside the batch, we have both classes
# So we do .sample(frac=1.0) to achieve kind of "uniform randomness"

from datasets import Dataset
all_responses = rlhf_df['toxic_response'].values + rlhf_df['non_toxic_response'].values
full_df = pd.DataFrame(all_responses, columns=['comment_text']).sample(frac=1.0)
toxic_for_rlhf = Dataset.from_pandas(full_df)

In [ ]:
sample_length = trl.core.LengthSampler(2, 8)

In [ ]:
# This method creates the query inside the dataset and will be used in TRL

def select_query_and_tokenize(sample):
    query_ids = main_tokenizer.encode(sample["comment_text"])[: sample_length()]
    sample["query"] = main_tokenizer.decode(query_ids)
    sample["input_ids"] = query_ids
    return sample

toxic_for_rlhf = toxic_for_rlhf.map(select_query_and_tokenize, batched=False)
toxic_for_rlhf.set_format(type="torch")

Map:   0%|          | 0/5009 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1204 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
training_args = trl.PPOConfig(
    model_name=main_model.config._name_or_path,
    gradient_accumulation_steps=1,
    learning_rate=1.41e-5,
    batch_size=8,
    mini_batch_size=8,
    ppo_epochs=4,                 # PPO performs this many updates per training batch
)

ppo_trainer = trl.PPOTrainer(training_args, model=main_model.model, tokenizer=main_tokenizer,
    dataset=toxic_for_rlhf, data_collator=lambda data: dict((key, [d[key] for d in data]) for key in data[0]))

In [ ]:
# Here we used our trained reward model to process batch of texts
# The signal from the reward model will push (align) our model with the direction we want to achieve (toxic/non-toxic)
from typing import List
def compute_reward(texts: List[str]) -> torch.Tensor:
  inputs = reward_tokenizer(texts, truncation=True, padding=True, return_tensors='pt').to(device)
  with torch.no_grad():
    return reward_model(**inputs).logits[:, 0]

In [ ]:
from tqdm.auto import tqdm
max_steps = 200   # can be insufficient for some tasks - watch your learning curves
generation_kwargs = dict(
    min_length=-1, max_new_tokens=128, do_sample=True, top_k=0, top_p=1.0, pad_token_id=main_tokenizer.eos_token_id)
#                                  ^-- task-specific parameter!
with tqdm(enumerate(ppo_trainer.dataloader), total=max_steps) as progressbar:
  # note: ppo_trainer.dataloader is just a regular dataloader of queries, no RL-specific magic :)
  for epoch, batch in progressbar:
    if epoch >= max_steps:
        break

    # Rollout stage: generate continuations from batch queries using main_model
    response_tensors = ppo_trainer.generate(batch['input_ids'], **generation_kwargs)
    # ^-- list of tensors of token ids from main model tokenizer

    # de-tokenize responses to strings (since reward model uses a different tokenizer)
    batch["response"] = [main_tokenizer.decode(response.squeeze()) for response in response_tensors]
    # note: response_tensors already contain query tokens, so we don't need to add queries manually.
    # This may not be true for other tasks: check this manually by viewing batch["response"] and batch["query"]


    # Evaluation stage
    rewards = compute_reward(batch['response'])

    # Update stage
    stats = ppo_trainer.step(batch['input_ids'], response_tensors, list(rewards.split(1)))
    stats['rewards/mean'] = rewards.mean().item()

    print("-" * 30, 'STEP', epoch, '-' * 30)
    print(f'rewards/mean:\t{stats["rewards/mean"]:.9f}\t<---- average reward over this batch (higher=better, noisy)')
    print(f'ppo/returns/mean:\t{stats["ppo/returns/mean"]:.9f}\t<---- model-estimated average discounted reward')
    print(f'objective/kl:\t{stats["objective/kl"]:.9f}\t<---- how far we are from the original model (regularizer)')
    print()

    ppo_trainer.log_stats(stats, batch, list(rewards.split(1)))

  0%|          | 0/200 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


------------------------------ STEP 0 ------------------------------
rewards/mean:	6.422979355	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.738241911	<---- model-estimated average discounted reward
objective/kl:	0.000000000	<---- how far we are from the original model (regularizer)

------------------------------ STEP 1 ------------------------------
rewards/mean:	7.361931801	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.886201978	<---- model-estimated average discounted reward
objective/kl:	-0.025272515	<---- how far we are from the original model (regularizer)

------------------------------ STEP 2 ------------------------------
rewards/mean:	6.649924755	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.843769908	<---- model-estimated average discounted reward
objective/kl:	-0.069572121	<---- how far we are from the original model (regularizer)

------------------------------ STEP 3 ---

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1279: UserWarning: KL divergence is starting to become negative: -1.16 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


------------------------------ STEP 18 ------------------------------
rewards/mean:	7.531183243	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	1.232703924	<---- model-estimated average discounted reward
objective/kl:	-1.159685612	<---- how far we are from the original model (regularizer)



...

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1279: UserWarning: KL divergence is starting to become negative: -1.42 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


------------------------------ STEP 33 ------------------------------
rewards/mean:	5.386047363	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	1.178748250	<---- model-estimated average discounted reward
objective/kl:	-1.418952346	<---- how far we are from the original model (regularizer)



/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1279: UserWarning: KL divergence is starting to become negative: -1.84 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


------------------------------ STEP 34 ------------------------------
rewards/mean:	6.142684937	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	1.454188585	<---- model-estimated average discounted reward
objective/kl:	-1.837672114	<---- how far we are from the original model (regularizer)



KeyboardInterrupt: 

In [ ]:
# Comment this if you don't want to save weights
save_lora_layers_and_embeddings(main_model, 'rlhf_model.pt')

In [ ]:
# Here we process the same prefixes and save them to RLHF outputs
rlhf_outputs = prefix_generation(prefixes, main_model.model, main_tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
import json
filename = 'rlhf_outputs.json'

with open(filename, 'w') as file:
    json.dump(rlhf_outputs, file)

### Task 2.4. Compart the results!
*(1 point)*

If we look at DPO- and RLHF-tuned models, they both generate more or less texts. However, DPO was much easier to train - we only had to train it once. Whereas for RLHF we had to do the Reward Modeling first and then use it for fine tuning of the main LLM.

Now, run everything and tell us what you think about the results

In [ ]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PREFIX</th>
    <th style="text-align: center; border:1px solid black">PRETRAIN</th>
    <th style="text-align: center; border:1px solid black">SFT</th>
    <th style="text-align: center; border:1px solid black">RLHF</th>
    <th style="text-align: center; border:1px solid black">DPO</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:20%; border:1px solid black"><pre align="left">{}</pre></td>

  </tr>'''

rows = []

for i, prefix in enumerate(prefixes):
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prefix, pre_train_outputs[i], sft_outputs[i], dpo_outputs[i], rlhf_outputs[i]))

display(HTML(table_template.format('\n'.join(rows))))

PREFIX,PRETRAIN,SFT,RLHF,DPO
`What`,"What is the best way to take photos? For me I prefer to take pictures from a tripod. I normally take pictures using a tripod with an adapter to adapt it to my camera (such as a Leica). It works well though I found that the first time. I did the photo tutorial, but when I tried it","What would you do with the time?"" What about the money? It's not enough to go the normal avenue of getting a lawyer, but just the thought of going to school is so great. ""Don't you mean I don't have that money?"" That would be nice, I think,","What was also in the early days of that campaign, I am told, were allegations that he had a mistress to act as his chief of staff; or a mistress named Laura, whose existence had never been disclosed by Mrs. Clinton. It is hard to take a little scandal to run a campaign. And that was a scandal","What's the deal? The original goal of the Kickstarter campaign was to fund the development of a new game based on the ""The Game That Will Never Die"" series, to be released for PC. Due to the incredible response to the Kickstarter, the developer is now bringing a modernised and improved version of this game"
`Who are you`,"Who are you? What are you doing here? Do you have any intention of stealing my life? Oh, my God, I'm your own nightmare in here. You're so stupid, you can't even see my mind. There must be some mistake. Let me look inside you. You're all the way inside me! I","Who are you doing this for?"" ""I… I don't know, but I was looking for something like that, but I do know of some that are called ""magic stones, crystal discs!"" I was just about to try one. I'm going to buy some for a friend of mine at work!"" """,Who are you from? I see many names on this list. What is your main interest? What do you do for a living? Which of the following would you like to find as your favorite? How do you spend your weekends? What is the most rewarding thing you do when you finish your,"Who are you taking out to?"" ""Well, if you call someone out you call me."" – ""I'm just taking out,"" said the girl. … There it is, the very definition of what a ""nice guy"" or ""nice girl"" is. It is, without question, a thing. To"
`Give it to`,"Give it to the other guys,' she told him. The three of them left, heading off into the night for another time. They were supposed to go back in a day or two. Now they were gone for a week. A few days later, they called each other again, and this time, the new","Give it to us all?"" I don't know, maybe I said it to my parents and they said, ""We will take it and pass it on to the government,"" but no more than that. So we were at the top of our field and we did, like, 2,000 cycles. So we were doing","Give it to people who've earned most of their living by using the internet and not writing or working with computers."" ""When I started, I used the internet a lot, as a way to keep in touch with friends, but now I've moved from that to more productive modes like word processing."" A computer helps","Give it to me, And I'll take you home with me Let it go I'd never knew you, We'd been so far apart But the night I kissed you in the car, The love I gave you last night, And the night I gave you a"
"`Hello, how is`","Hello, how is your day going? Are you at it all? No, just lying there with a ball on your head, and watching your favourite anime characters. I'm not lying, it IS very entertaining when a story has a little bit of logic to it. In some ways, it can be kind of depressing and you","Hello, how is everyone going? How are you, Dad? I'm well, all I need is some ice cream and some wine. I need to do some things now I'm getting a little bit sleepy after my long trip. Let me talk to you about this later. The children are not allowed to be outside during daylight hours.","Hello, how is your week going? How could I help you?"" ""It is going well, thanks."" Then the woman said, ""I'm sorry. But some people are just shy."" Her tone was so cold I couldn't help but say,